In [1]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_name = 'spvae'

data_file = f'../{data_name}_vectors_drags_lifts.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)

#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

/home/xli/anaconda3/envs/surrogate_autogluon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_121,dim_122,dim_123,dim_124,dim_125,dim_126,dim_127,dim_128,drag,lift
406,-0.000353,0.938900,0.730082,0.001080,-0.723818,-0.000713,-0.363452,1.363965,0.000757,1.158052,...,0.000228,0.000479,0.000233,-0.000070,-1.169997,-0.630202,-0.001056,0.000770,0.395,0.425
54,0.000310,-0.932797,0.025703,0.000571,-1.087611,0.000181,0.035749,-0.290552,-0.030080,0.761045,...,0.000569,-0.001818,-0.000403,0.000658,0.668129,2.250648,-0.010414,0.001804,0.645,0.885
241,-0.000317,-1.111809,-1.151887,-0.000197,-0.180751,0.000557,-0.561294,0.094284,0.002247,0.794197,...,0.000313,-0.000366,-0.000735,0.001392,-0.409571,-2.031675,-0.001192,0.000625,0.446,0.756
952,0.000499,0.457167,-1.479584,0.001333,-0.577155,0.000301,-0.905196,-1.337308,-0.001215,-1.755122,...,0.000282,-0.000194,-0.000080,-0.000260,-0.697985,-1.125502,-0.000115,0.000077,0.384,0.177
337,0.000667,0.547938,0.491766,0.000550,0.434486,0.000730,3.182191,-0.781295,-0.002309,1.794750,...,0.000391,-0.000374,-0.000856,-0.000261,0.332565,0.792273,-0.000045,-0.000629,0.381,0.074


In [2]:
label = 'lift'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    837.000000
mean       0.378389
std        0.238650
min        0.002000
25%        0.184000
50%        0.344000
75%        0.540000
max        0.988000
Name: lift, dtype: float64


In [3]:
import os
save_path = f'./agModels-{data_name}_{label}'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

bag_folds = 5 #suggestion range [5, 10]
bag_sets = 3 #suggestion range [1, 20]
stack_levels = 3 #suggestion range [0, 3]
metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, 
                                                                                  presets='best_quality', 
                                                                                  auto_stack="True", 
                                                                                  num_bag_folds=bag_folds, 
                                                                                  num_bag_sets=bag_sets,
                                                                                  num_stack_levels=stack_levels,
                                                                                  verbosity=4)

Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': 'True',
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'verbosity': 4}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': 'True',
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=5, num_bag_sets=3
Saving ./agModels-spvae_lift/learner.pkl
Saving ./agModels-spvae_lift/predictor.pkl

In [4]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_120,dim_121,dim_122,dim_123,dim_124,dim_125,dim_126,dim_127,dim_128,drag
1046,0.000482,-1.632118,0.521768,0.000057,-0.524977,0.001181,-1.053361,-0.710915,-0.014259,-0.247215,...,0.940937,-0.000201,-0.000543,-0.000332,-0.000086,-0.250152,0.810739,-0.005433,0.000972,0.527
657,0.000334,-0.717807,0.156846,0.000663,-0.597166,0.000386,-2.051708,0.732814,-0.017638,-0.092468,...,-1.009466,-0.000286,-0.001223,-0.000568,0.000159,0.285532,0.058386,-0.006653,0.000682,0.363
127,-0.001045,0.985687,0.333722,-0.000779,0.795979,0.000015,0.222498,1.193969,-0.002246,0.801656,...,1.449672,0.000892,-0.000499,-0.000412,0.001091,0.325954,0.986142,-0.002773,0.000732,0.316
951,-0.000010,-1.110299,-1.179036,0.001446,0.173944,0.000330,0.236702,-0.755380,-0.002542,-1.876672,...,-2.466808,0.000105,0.000189,0.001414,-0.000600,0.408066,1.761171,-0.000150,0.001450,0.413
134,-0.000103,2.300475,-0.223271,0.001451,0.734430,0.000361,0.682707,-0.307900,-0.000511,2.187333,...,-0.736073,-0.000425,0.000223,0.000188,-0.001001,-1.714110,-0.823680,-0.000549,0.000145,0.495


In [5]:
# %%capture log_output
# %config InlineBackend.figure_format = 'retina'
# %config Application.log_level = 'DEBUG'
# %config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

# with open('./output_all_parts.log', 'w') as f:
#     f.write(log_output.stdout)

Loading: ./agModels-spvae_lift/predictor.pkl
Loading: ./agModels-spvae_lift/learner.pkl
Loading: ./agModels-spvae_lift/models/trainer.pkl
Loading: ./agModels-spvae_lift/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/WeightedEnsemble_L2/model.pkl
Evaluation: root_mean_squared_error on test data: -0.20168884608145124
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -0.20168884608145124,
    "mean_squared_error": -0.04067839063366733,
    "mean_absolute_error": -0.1606778474240076,
    "r2": 0.3141399254437569,
    "pearsonr": 0.5644223048234288,
    "median_absolute_error": 

0.4557919204235077
0.3015097379684448
0.3186054527759552
0.5477826595306396
0.2041417956352234
0.3339441418647766
0.3311759829521179
0.6231769919395447
0.2991415560245514
0.2853679060935974
0.38707196712493896
0.2932627499103546
0.3583693206310272
0.3889940679073334
0.3115076422691345
0.44033733010292053
0.4699525833129883
0.29327672719955444
0.6910973787307739
0.3827074468135834
0.2340696007013321
0.3979591727256775
0.2146483212709427
0.4594993591308594
0.23672713339328766
0.5212499499320984
0.46359896659851074
0.38515812158584595
0.23590992391109467
0.537783682346344
0.31648018956184387
0.31850746273994446
0.49210143089294434
0.2870692312717438
0.32718726992607117
0.5245308876037598
0.34624963998794556
0.1741613894701004
0.26120734214782715
0.25296899676322937
0.3214963376522064
0.5030612945556641
0.28474047780036926
0.2726459503173828
0.4573585093021393
0.5720674395561218
0.2700747549533844
0.3256475329399109
0.562079668045044
0.5304001569747925
0.4241310954093933
0.3134573400020599

/home/xli/anaconda3/envs/surrogate_autogluon/lib/python3.10/site-packages/autogluon/core/utils/plots.py:138: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')
Loading: ./agModels-spvae_lift/models/KNeighborsUnif_BAG_L1/model.pkl


{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN', 'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN', 'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB', 'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB', 'RandomForestMSE_BAG_L1': 'StackerEnsembleModel_RF', 'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost', 'ExtraTreesMSE_BAG_L1': 'StackerEnsembleModel_XT', 'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular', 'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost', 'NeuralNetTorch_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch', 'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB', 'WeightedEnsemble_L2': 'WeightedEnsembleModel', 'LightGBMXT_BAG_L2': 'StackerEnsembleModel_LGB', 'LightGBM_BAG_L2': 'StackerEnsembleModel_LGB', 'RandomForestMSE_BAG_L2': 'StackerEnsembleModel_RF', 'CatBoost_BAG_L2': 'StackerEnsembleModel_CatBoost', 'ExtraTreesMSE_BAG_L2': 'StackerEnsembleModel_XT', 'NeuralNetFastAI_BAG_L2': 'StackerEnsembleModel_NNFastAiTabular', 'XGBoost_BAG_L2': 'S

Loading: ./agModels-spvae_lift/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/WeightedEnsemble_L2/model.pkl
Loading: ./agModels-spvae_lift/models/LightGBMXT_BAG_L2/model.pkl
Loading: ./agModels-spvae_lift/models/LightGBM_BAG_L2/model.pkl
Loading: ./agModels-spvae_lift/models/RandomForestMSE_BAG_L2/model.pkl
Loading: ./agModels-spvae_lift/models/CatBoost_BAG_L2/

                     model  score_test  score_val  pred_time_test  \
0     ExtraTreesMSE_BAG_L2   -0.199295  -0.199386       15.688545   
1          LightGBM_BAG_L2   -0.200641  -0.199912       17.675472   
2      WeightedEnsemble_L3   -0.201105  -0.198121       20.812598   
3        LightGBMXT_BAG_L2   -0.201636  -0.198919       17.648709   
4      WeightedEnsemble_L2   -0.201689  -0.197644       10.487692   
5   RandomForestMSE_BAG_L2   -0.201873  -0.201663       15.732591   
6     ExtraTreesMSE_BAG_L3   -0.201899  -0.202153       24.266255   
7      WeightedEnsemble_L4   -0.202254  -0.200980       24.468590   
8     LightGBMLarge_BAG_L2   -0.202528  -0.202431       18.318760   
9          CatBoost_BAG_L3   -0.203330  -0.201557       24.299747   
10  RandomForestMSE_BAG_L3   -0.203620  -0.206302       24.334887   
11          XGBoost_BAG_L2   -0.203921  -0.202055       15.788471   
12         CatBoost_BAG_L2   -0.203977  -0.199812       15.704608   
13  RandomForestMSE_BAG_L4   -0.20

In [6]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  regression
AutoGluon identified the following types of features:
('float', []) : 129 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]


In [7]:
train_data_pred = predictor.predict(train_data)
test_data_pred = predictor.predict(test_data)

import numpy as np
#save np array y_train_hat to a csv file
np.savetxt(f'./{data_name}_vectors_y_test_hat_{label}.csv', test_data_pred, delimiter=',')
np.savetxt(f'./{data_name}_vectors_y_train_hat_{label}.csv', train_data_pred, delimiter=',')

Loading: ./agModels-spvae_lift/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/WeightedEnsemble_L2/model.pkl
Loading: ./agModels-spvae_lift/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-spvae_lift/models/WeightedEnsemble_L2/model.pkl
